### Construction of a stemness index (mRNAsi) model using the OCLR algorithm

In [ ]:
library(synapser)

In [ ]:
setwd("/media/bio/Disk/Research Data/EBV/omicverse")

In [ ]:
# Register an account on https://www.synapse.org/ and log in using your Personal Access Token (also known as Synapse Auth Token)
# install.packages("synapser", repos=c("http://ran.synapse.org", "https://cloud.r-project.org"))
synLogin(email = "XXXX@gmail.com", silent=FALSE, authToken="XXXX", cache_client=TRUE)
# After successful login, use the synGet function to download the RNA expression data
synRNA <- synGet( "syn2701943", downloadLocation = "/media/bio/Disk/Research Data/EBV/omicverse/Dataset/mRNAsi" )

In [ ]:
library(tidyverse)
exp <- read_delim(file = synRNA$path) %>%
  separate(col = "tracking_id", sep = "\\.", into = c("Ensembl_ID", "suffix")) %>%
  dplyr::select(-suffix) %>%
  column_to_rownames("Ensembl_ID") %>%
  as.matrix()
exp[1:3,1:3]

In [ ]:
library(org.Hs.eg.db)
unimap <- mapIds(
  org.Hs.eg.db, keys = rownames(exp), keytype = "ENSEMBL", 
  column = "SYMBOL", multiVals = "filter")
data.exp <- exp[names(unimap),]
rownames(data.exp) <- unimap

In [ ]:
library(dplyr)
synMeta <- synTableQuery("SELECT UID, Diffname_short FROM syn3156503")
metaInfo <- synMeta$asDataFrame()
if (!is.null(rownames(metaInfo))) {
  rownames(metaInfo) <- NULL 
}
metaInfo <- metaInfo %>%
  dplyr::select(UID, Diffname_short) %>%
  filter(!is.na(Diffname_short))
metaInfo <- column_to_rownames(metaInfo, "UID")
X <- data.exp
y <- metaInfo[colnames(X), ]
names(y) <- colnames(X)
head(y)

In [ ]:
library(gelnet)
X <- data.exp
m <- apply(X, 1, mean)
X <- X - m
sc <- which(y == "SC")
X.sc <- X[, sc]
X.or <- X[, -sc]
model.RNA <- gelnet(t(X.sc), NULL, 0, 1)
head(model.RNA$w)
save(X, y, model.RNA, file = "/media/bio/Disk/Research Data/EBV/omicverse/Dataset/mRNAsi/model.rda")